In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [2]:
#######################################################
# Model Test on TestSet
# 11/09/2019 
#
#######################################################
import numpy as np
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import os
import cv2
from math import floor
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from ISY5002_CA2_02_ModelDefinitions import createModel, my_preprocess
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as VGG16_preprocess_input
from tensorflow.keras.applications.resnet50 import preprocess_input as ResNet50_preprocess_input
from tensorflow.keras.applications.inception_v3 import preprocess_input as InceptionV3_preprocess_input

# import sys

# defining global variables
DEBUG_MODE = False
optmz       = 'adam'    # optimizers.RMSprop(lr=0.0001)

def getAccuracyOnTestdata(index, size):

    modelGo = createModel(size,size,3, index)
    modelGo.load_weights('./CA2_' + str(index)+ "_" + str(size) + ".hdf5")
    modelGo.compile(loss='categorical_crossentropy', 
                optimizer=optmz, 
                metrics=['accuracy'])

    preprocessing_fn = my_preprocess
    if (index > 90):
        # Using pretrained datasets - have to use their preprocess functions
        if (index == 90):
            preprocessing_fn = VGG16_preprocess_input
        elif (index == 91):
            preprocessing_fn = ResNet50_preprocess_input
        elif (index == 92):
            preprocessing_fn = InceptionV3_preprocess_input
    #     datagen = ImageDataGenerator(preprocessing_function = preprocessing_fn)
    # else:
    #     datagen = ImageDataGenerator(rescale=1.0/255)
    datagen = ImageDataGenerator(preprocessing_function = preprocessing_fn)

    test_it = datagen.flow_from_directory('./Resized/test/', class_mode='categorical', target_size=(size, size), batch_size=32, color_mode='rgb', shuffle = False)
    predicts = modelGo.predict_generator(test_it)
    labelname = list(test_it.class_indices.keys())
    predout = np.argmax(predicts, axis = 1)

    testScores = metrics.accuracy_score(test_it.labels, predout)
    confusion = metrics.confusion_matrix(test_it.labels, predout)

    print("Best accuracy (on testing dataset): %.2f%%" % (testScores*100)) 
    print(metrics.classification_report(test_it.labels, predout, target_names=labelname, digits=4 )) 
    print(confusion)    

def main():
    # --------------------------
    # ------ CHANGE THESE ------
    # get accuracy for XX
    getAccuracyOnTestdata(90, 224)
    # --------------------------
    
main()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Found 780 images belonging to 3 classes.
Best accuracy (on testing dataset): 96.92%
              precision    recall  f1-score   support

        bees     0.9771    0.9846    0.9808       260
 butterflies     0.9727    0.9577    0.9651       260
       moths     0.9580    0.9654    0.9617       260

    accuracy                         0.9692       780
   macro avg     0.9693    0.9692    0.9692       780
weighted avg     0.9693    0.9692    0.9692       780

[[256   2   2]
 [  2 249   9]
 [  4   5 251]]
